In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/fraud-detection-bank-dataset-20k-records-binary/fraud_detection_bank_dataset.csv")

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df = df.drop("Unnamed: 0",axis = 1)
df

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
def std_norm(df, column):
    c = df[column]
    if(c.std()!=0 and column != "targets"):
        df[column] = (c - c.mean())/c.std()

for i in df.columns:
    std_norm(df,i)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.corr()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize= (10,10,))
sns.heatmap(df.corr())

In [ ]:
Y = df["targets"]
X = df.drop("targets",axis = 1)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(X)

In [ ]:
pca.transform(X)

In [ ]:
import plotly.express as px
from sklearn.decomposition import PCA

pca = PCA()
components = pca.fit_transform(X)
labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

fig = px.scatter_matrix(
    components,
    labels=labels,
    dimensions=range(4),
    color=Y
)
fig.update_traces(diagonal_visible=False)
fig.show()

In [ ]:
#X = pca.transform(X)

In [ ]:
from sklearn.model_selection._split import train_test_split
from random import randint
X_train , X_test , y_train , y_test = train_test_split(X,Y,test_size = 0.3,random_state=12)

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

n_estimators = 10
max_depth = 3

Logistic_Regression = LogisticRegression(solver="liblinear",class_weight={1:2}, random_state=10)
Logistic_Regression.fit(X_train, y_train)

random_forest = RandomForestClassifier(
    n_estimators=n_estimators, max_depth=max_depth, random_state=10)
random_forest.fit(X_train, y_train)

gradient_boosting = GradientBoostingClassifier(
    n_estimators=n_estimators, max_depth=max_depth, random_state=10)
_ = gradient_boosting.fit(X_train, y_train)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import plot_roc_curve

fig, ax = plt.subplots()

models = [
    ("RF", random_forest),
    ("LR", Logistic_Regression),
    ("GBDT", gradient_boosting),
]

model_displays = {}
for name, model in models:
    model_displays[name] = plot_roc_curve(
        model, X_test, y_test, ax=ax, name=name)
_ = ax.set_title('ROC curve')

In [ ]:
from sklearn.metrics import precision_score
for name,model in models:
    precision = precision_score(y_test,model.predict(X_test) )
    print("precision of {}:{}".format(name,precision))